## Exploratory Analysis



In [43]:
%load_ext autoreload
%autoreload 2
import os
import pandas as pd

train_path = "../../data/sentiment/training_set_sentipolc16.csv"
test_path = "../../data/sentiment/test_set_sentipolc16_gold2000.csv"

# Read SENTIPOLC16 data
# Take care that data is separated by commas and double quotes
train_df = pd.read_csv(train_path, sep=",", quotechar='"')
test_df = pd.read_csv(test_path, sep=",", quotechar='"')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ParserError: Error tokenizing data. C error: Expected 9 fields in line 110, saw 10


In [32]:
#Open test set and check what's going on


In [44]:
a = [1,2,3,4,5]

a.pop()

5

In [61]:
# Try with csv reader
import csv


lines = list(open("../../data/sentiment/test_set_sentipolc16_gold2000.csv", "r"))

reader = csv.reader(lines, quotechar='"', delimiter=',',
                     quoting=csv.QUOTE_ALL, skipinitialspace=True)

test_data = []

for row in reader:
    while len(row) != 9:
        # Join last one with commas
        last = row.pop()
        before_last = row.pop()

        row.append(before_last + "," + last)
    test_data.append(row)


In [62]:
from collections import Counter
Counter(len(t) for t in test_data)

Counter({9: 2000})

In [64]:
df_test = pd.DataFrame(test_data, columns=["idtwitter","subj","opos","oneg","iro","lpos","lneg","top","text"])

In [81]:
from datasets import DatasetDict, Dataset, Value, Features
from datasets import Value
features = Features({
    k:Value('string') if k == "text" else Value('int64') for k in train_df.columns
})

ds = DatasetDict(
    train=Dataset.from_pandas(train_df, features=features), 
    test=Dataset.from_pandas(df_test, features=features)
)

In [82]:
ds.push_to_hub("pysentimiento/it_sentipolc16", private=True)

Pushing split train to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/8 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

Pushing split test to the Hub.


Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

In [83]:
train_df.columns

Index(['idtwitter', 'subj', 'opos', 'oneg', 'iro', 'lpos', 'lneg', 'top',
       'text'],
      dtype='object')

See http://www.di.unito.it/~tutreeb/sentipolc-evalita16/sentipolc-guidelines2016UPDATED130916.pdf for the guidelines.